<a href="https://colab.research.google.com/github/IoThang/AI/blob/main/FINALL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import Dropout
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.stem.porter import PorterStemmer

In [ ]:
df=pd.read_csv('training.csv') 
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
df=pd.read_csv('training.csv')  # Read file
df=df.dropna()###Drop Nan Values  
X=df.drop('sentiment',axis=1) # Seperate column: Text column
y=df['sentiment'] # Pos/Neg column

In [ ]:
X.head(10)

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."
5,"Probably my all-time favorite movie, a story o..."
6,I sure would like to see a resurrection of a u...
7,"This show was an amazing, fresh & innovative i..."
8,Encouraged by the 1 comments about this film o...
9,If you like original gut wrenching laughter yo...


In [ ]:
y.head(10)

0    1
1    1
2    1
3    0
4    1
5    1
6    1
7    0
8    0
9    1
Name: sentiment, dtype: int64

In [ ]:
voc_size=5000

In [ ]:
messages=X.copy()
messages['review'][1]
messages.reset_index(inplace=True)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1952,
  2601,
  4273,
  4333,
  4147,
  545,
  692,
  4095,
  720,
  2613,
  4407,
  4407,
  216,
  4553,
  3574,
  4147,
  3049,
  3856,
  2141,
  4752,
  2743,
  4095,
  4985,
  2683,
  2235,
  308,
  1414,
  1142,
  321,
  308,
  560,
  4251,
  2248,
  80,
  2942,
  4752,
  1184,
  2076,
  3181,
  4985,
  4407,
  4407,
  3306,
  4147,
  281,
  2548,
  280,
  3139,
  1949,
  1070,
  2040,
  2958,
  3566,
  2772,
  1778,
  3202,
  1743,
  2138,
  2605,
  4267,
  1697,
  4624,
  1625,
  2102,
  795,
  3612,
  706,
  1778,
  2692,
  2823,
  4123,
  1753,
  3843,
  4343,
  4450,
  2996,
  392,
  3565,
  2341,
  4462,
  4833,
  2468,
  4182,
  4886,
  562,
  3774,
  913,
  4407,
  4407,
  3530,
  4519,
  45,
  1410,
  308,
  2279,
  3097,
  1777,
  308,
  2535,
  4021,
  556,
  4791,
  4203,
  683,
  3223,
  4021,
  3229,
  4021,
  2519,
  4147,
  2910,
  3762,
  216,
  545,
  4476,
  2174,
  3574,
  2015,
  2147,
  4519,
  4353,
  4333,
  4305,
  596,
  4147,
  3547,
  4519,
  795,
  1

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[2138  904 2279 ...  406 1932 2015]
 [2707 4786 3047 ... 4828 1975  741]
 [2999 4482 4653 ... 2683  972 4152]
 ...
 [2330 2263 2382 ... 1651 1967 4450]
 [3445  405 3088 ... 2865  110 2752]
 [2432 1212 1952 ...  508 2846 2895]]


In [ ]:
embedded_docs[0]

array([2138,  904, 2279, 1644, 1734, 4434, 2138, 4733, 4333, 4147,  913,
       3670, 4974, 1981, 2379, 4727,  197,  406, 1932, 2015], dtype=int32)

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 266,601
Trainable params: 266,601
Non-tr

In [ ]:
len(embedded_docs),y.shape

(50000, (50000,))

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((50000, 20), (50000,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
547/547 [==============================] - 32s 51ms/step - loss: 0.5211 - accuracy: 0.7349 - val_loss: 0.4752 - val_accuracy: 0.7740
Epoch 2/20
547/547 [==============================] - 27s 50ms/step - loss: 0.4411 - accuracy: 0.7940 - val_loss: 0.4745 - val_accuracy: 0.7707
Epoch 3/20
547/547 [==============================] - 30s 55ms/step - loss: 0.4109 - accuracy: 0.8113 - val_loss: 0.4747 - val_accuracy: 0.7691
Epoch 4/20
547/547 [==============================] - 28s 52ms/step - loss: 0.3845 - accuracy: 0.8256 - val_loss: 0.5075 - val_accuracy: 0.7647
Epoch 5/20
547/547 [==============================] - 30s 55ms/step - loss: 0.3608 - accuracy: 0.8342 - val_loss: 0.5205 - val_accuracy: 0.7643
Epoch 6/20
547/547 [==============================] - 30s 55ms/step - loss: 0.3359 - accuracy: 0.8468 - val_loss: 0.5338 - val_accuracy: 0.7597
Epoch 7/20
547/547 [==============================] - 28s 52ms/step - loss: 0.3127 - accuracy: 0.8566 - val_loss: 0.5660 - val_accuracy:

In [ ]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")
accuracy_score(y_test,y_pred)

469/469 [==============================] - 4s 8ms/step


0.7336666666666667

In [ ]:
print(y_test[:10])
print(y_pred[:10])

[1 1 0 1 0 1 1 1 0 0]
[[0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]]


In [ ]:
def runtest(text):
  ps = PorterStemmer()
  corpus1 = []
  for i in range(0, len(text)):
     
      review = re.sub('[^a-zA-Z]', ' ', text)
      review = review.lower()
      review = review.split()
      
      review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
      review = ' '.join(review)
      corpus1.append(review)

  onehot_repr1=[one_hot(words,voc_size)for words in corpus1] 
  embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=20)
  test = np.array(embedded_docs1)
  y_pred=(model.predict(test) > 0.5).astype("int32")
  return y_pred[0]

In [ ]:
text = input('Enter Title: ')
x = runtest(text)

if (x==1):
  print('a positive comment !')
else:
  print('a negative comment !')

Enter Title: If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it.<br /><br />Great Camp!!!
6/6 [==============================] - 0s 9ms/step
a positive comment !


In [ ]:
model.save("/content/drive/MyDrive/final/finalProject.h5")